In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rcParams['figure.figsize'] = (18, 8)

In [44]:
import warnings

warnings.filterwarnings('ignore')

In [45]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
gender_submission = pd.read_csv('gender_submission.csv')

In [46]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# Sex

In [47]:
train['Sex_clean'] = train['Sex'].astype('category').cat.codes
test['Sex_clean'] = test['Sex'].astype('category').cat.codes

# Embarked

In [48]:
train['Embarked'].isnull().sum()

2

In [49]:
test['Embarked'].isnull().sum()

0

In [50]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [51]:
train['Embarked'].fillna('S', inplace=True)

In [52]:
train['Embarked'].isnull().sum()

0

In [53]:
train['Embarked_clean'] = train['Embarked'].astype('category').cat.codes
test['Embarked_clean'] = test['Embarked'].astype('category').cat.codes

# Family

In [54]:
train['Family'] = 1 + train['SibSp'] + train['Parch']
test['Family'] = 1 + test['SibSp'] + test['Parch']

In [55]:
train['Solo'] = (train['Family'] == 1)
test['Solo'] = (test['Family'] == 1)

# Fare

In [56]:
train['FareBin'] = pd.qcut(train['Fare'], 5)
test['FareBin'] = pd.qcut(test['Fare'], 5)
train['Fare_clean2'] = train['FareBin'].astype('category').cat.codes
test['Fare_clean2'] = test['FareBin'].astype('category').cat.codes

In [57]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_clean,Embarked_clean,Family,Solo,FareBin,Fare_clean2
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,2,2,False,"(-0.001, 7.854]",0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,0,2,False,"(39.688, 512.329]",4
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,2,1,True,"(7.854, 10.5]",1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,2,2,False,"(39.688, 512.329]",4
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,2,1,True,"(7.854, 10.5]",1


# Title

In [58]:
train['Title'] = train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

In [59]:
test['Title'] = test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

In [60]:
train['Title'] = train['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')

In [61]:
train['Title'].value_counts()

Mr        517
Miss      182
Mrs       125
Master     40
Other      23
Mlle        2
Mme         1
Ms          1
Name: Title, dtype: int64

In [62]:
train['Title'] = train['Title'].replace('Mlle', 'Miss')
train['Title'] = train['Title'].replace('Ms', 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')

In [63]:
train['Title'].value_counts()

Mr        517
Miss      185
Mrs       126
Master     40
Other      23
Name: Title, dtype: int64

In [64]:
test['Title'] = test['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')

In [65]:
test['Title'] = test['Title'].replace('Mlle', 'Miss')
test['Title'] = test['Title'].replace('Ms', 'Miss')
test['Title'] = test['Title'].replace('Mme', 'Mrs')

In [66]:
test['Title'].value_counts()

Mr        240
Miss       79
Mrs        72
Master     21
Other       6
Name: Title, dtype: int64

In [67]:
train['Title_clean'] = train['Title'].astype('category').cat.codes
test['Title_clean'] = test['Title'].astype('category').cat.codes

# Age

In [68]:
train["Age"].fillna(train.groupby("Title")["Age"].transform("median"), inplace=True)
test["Age"].fillna(train.groupby("Title")["Age"].transform("median"), inplace=True)

In [69]:
train.loc[ train['Age'] <= 16, 'Age_claen'] = 0
train.loc[(train['Age'] > 16) & (train['Age'] <= 26), 'Age_clean'] = 1
train.loc[(train['Age'] > 26) & (train['Age'] <= 36), 'Age_clean'] = 2
train.loc[(train['Age'] > 36) & (train['Age'] <= 62), 'Age_clean'] = 3
train.loc[ train['Age'] > 62, 'Age_clean'] = 4

In [70]:
test.loc[ test['Age'] <= 16, 'Age_clean'] = 0
test.loc[(test['Age'] > 16) & (test['Age'] <= 26), 'Age_clean'] = 1
test.loc[(test['Age'] > 26) & (test['Age'] <= 36), 'Age_clean'] = 2
test.loc[(test['Age'] > 36) & (test['Age'] <= 62), 'Age_clean'] = 3
test.loc[ test['Age'] > 62, 'Age_clean'] = 4

In [71]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Sex_clean,Embarked_clean,Family,Solo,FareBin,Fare_clean2,Title,Title_clean,Age_claen,Age_clean
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,1,2,2,False,"(-0.001, 7.854]",0,Mr,2,NaN,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,0,0,2,False,"(39.688, 512.329]",4,Mrs,3,NaN,3.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,0,2,1,True,"(7.854, 10.5]",1,Miss,1,NaN,1.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,0,2,2,False,"(39.688, 512.329]",4,Mrs,3,NaN,2.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,1,2,1,True,"(7.854, 10.5]",1,Mr,2,NaN,2.0


# Fare

In [72]:
train["Fare"].fillna(train.groupby("Pclass")["Fare"].transform("median"), inplace=True)
test["Fare"].fillna(test.groupby("Pclass")["Fare"].transform("median"), inplace=True)

In [73]:
pd.qcut(train['Fare'], 4)

0       (-0.001, 7.91]
1      (31.0, 512.329]
2       (7.91, 14.454]
3      (31.0, 512.329]
4       (7.91, 14.454]
            ...       
886     (7.91, 14.454]
887     (14.454, 31.0]
888     (14.454, 31.0]
889     (14.454, 31.0]
890     (-0.001, 7.91]
Name: Fare, Length: 891, dtype: category
Categories (4, interval[float64, right]): [(-0.001, 7.91] < (7.91, 14.454] < (14.454, 31.0] < (31.0, 512.329]]

In [74]:
train.loc[ train['Fare'] <= 17, 'Fare_clean'] = 0
train.loc[(train['Fare'] > 17) & (train['Fare'] <= 30), 'Fare_clean'] = 1
train.loc[(train['Fare'] > 30) & (train['Fare'] <= 100), 'Fare_clean'] = 2
train.loc[ train['Fare'] > 100, 'Fare_clean'] = 3
train['Fare_clean'] = train['Fare_clean'].astype(int)

In [75]:
test.loc[ test['Fare'] <= 17, 'Fare_clean'] = 0
test.loc[(test['Fare'] > 17) & (test['Fare'] <= 30), 'Fare_clean'] = 1
test.loc[(test['Fare'] > 30) & (test['Fare'] <= 100), 'Fare_clean'] = 2
test.loc[ test['Fare'] > 100, 'Fare_clean'] = 3
test['Fare_clean'] = test['Fare_clean'].astype(int)

In [76]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Embarked_clean,Family,Solo,FareBin,Fare_clean2,Title,Title_clean,Age_claen,Age_clean,Fare_clean
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,2,2,False,"(-0.001, 7.854]",0,Mr,2,NaN,1.0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,0,2,False,"(39.688, 512.329]",4,Mrs,3,NaN,3.0,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,2,1,True,"(7.854, 10.5]",1,Miss,1,NaN,1.0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,2,2,False,"(39.688, 512.329]",4,Mrs,3,NaN,2.0,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,2,1,True,"(7.854, 10.5]",1,Mr,2,NaN,2.0,0


# Cabin

In [77]:
train['Cabin'].str[:1].value_counts()

C    59
B    47
D    33
E    32
A    15
F    13
G     4
T     1
Name: Cabin, dtype: int64

In [78]:
mapping = {
    'A' : 0,
    'B' : 1,
    'C' : 2,
    'D' : 3,
    'E' : 4,
    'F' : 5,
    'G' : 6,
    'T' : 7
}

In [79]:
train['Cabin_clean'] = train['Cabin'].str[:1]

In [80]:
train['Cabin_clean'] = train['Cabin_clean'].map(mapping)

In [81]:
train[['Pclass', 'Cabin_clean']].head(10)

,Pclass,Cabin_clean
0,3,NaN
1,1,2.0
2,3,NaN
3,1,2.0
4,3,NaN
5,3,NaN
6,1,4.0
7,3,NaN
8,3,NaN
9,2,NaN


In [82]:
train['Cabin_clean'] = train.groupby('Pclass')['Cabin_clean'].transform('median')

In [83]:
train['Cabin_clean'].head(10)

0    5.0
1    2.0
2    5.0
3    2.0
4    5.0
5    5.0
6    2.0
7    5.0
8    5.0
9    4.5
Name: Cabin_clean, dtype: float64

위와 같은 방법으로 할 경우 이미 자리 잡은 값들이 덮어 씌어짐
<br>따라서 아래와 같은 방법으로 없는 값들만 채워줘야함 (fillna)

In [84]:
train['Cabin_clean'].fillna(train.groupby('Pclass')['Cabin_clean'].transform('median'), inplace=True)

In [85]:
train['Cabin_clean'].head(10)

0    5.0
1    2.0
2    5.0
3    2.0
4    5.0
5    5.0
6    2.0
7    5.0
8    5.0
9    4.5
Name: Cabin_clean, dtype: float64

In [86]:
test['Cabin_clean'] = test['Cabin'].str[:1]
test['Cabin_clean'] = test['Cabin_clean'].map(mapping)
test['Cabin_clean'].fillna(test.groupby('Pclass')['Cabin_clean'].transform('median'), inplace=True)

# Feature & label

In [87]:
feature = [
    'Pclass',
    'SibSp',
    'Parch',
    'Sex_clean',
    'Embarked_clean',
    'Family',
    'Solo',
    'Title_clean',
    'Fare_clean',
    'Cabin_clean'
]

In [88]:
label = [
    'Survived',
]

# Model Selection

In [91]:
data = train[feature]
target = train[label]

In [92]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [93]:
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

In [94]:
x_train, x_test, y_train, y_test = train_test_split(data, target, random_state=0)

In [97]:
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

In [98]:
clf = SVC(random_state=0)
cross_val_score(clf, data, target, cv=k_fold, scoring='accuracy', ).mean()

0.8305118601747814

In [99]:
clf = RandomForestClassifier(n_estimators=180, max_depth=5, random_state=0)
cross_val_score(clf, data, target, cv=k_fold, scoring='accuracy', ).mean()

0.8159176029962547

In [101]:
clf = LGBMClassifier(n_estimators=120,
                    max_depth=6,
                    subsample=0.8,
                    colsample_bytree=0.8, random_state=0)
cross_val_score(clf, data, target, cv=k_fold, scoring='accuracy', ).mean()

0.8293757802746567

In [102]:
clf = XGBClassifier(n_estimators=90,
                   max_depth=5,
                   subsample=0.8,
                   colsample_bytree=0.8, random_state=0)
cross_val_score(clf, data, target, cv=k_fold, scoring='accuracy', ).mean()

[00:39:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:39:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:39:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:39:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

0.8248938826466917

# Make Prediction

In [103]:
x_train = train[feature]
x_test = test[feature]
y_train = train[label]

In [106]:
clf.fit(x_train, y_train)

[00:40:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8,
              enable_categorical=False, gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=90, n_jobs=12, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.8, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [107]:
pred = clf.predict(x_test)

In [109]:
gender_submission['Survivied'] = pred

In [110]:
gender_submission.to_csv('191111_random_forest.csv', index=False)

In [111]:
clf

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8,
              enable_categorical=False, gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=90, n_jobs=12, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.8, tree_method='exact', validate_parameters=1,
              verbosity=None)